
# 🌼 대규모 LLM을 활용한 지식 챗봇 개발(1) - 3차시(24.11.25)

---


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import numpy as np
import tensorflow as tf
import transformers
import pandas as pd

In [ ]:
comment=pd.read_excel('/content/drive/MyDrive/A_comment_train.xlsx',index_col=0)  # index_col=0: 0번 컬럼을 인덱스로 하겠다

In [ ]:
comment.head()

,data,label
0,재미는 있는데 시간이 짧은게 아쉽네요~,긍정
1,"OO 관련 내용은 우리 직원과는 거리가 멀었음, 특히, 사내에 홍보할 내용은 아니라고 봄",부정
2,스토리가 너무 딱딱해서 별로였음,부정
3,프로그램A 화이팅하세요!!,긍정
4,높은 곳에 올라가는 모습이 너무 위험해 보여요.,부정


In [ ]:
comment.isnull().sum()

,0
data,0
label,0


In [ ]:
comment['label'] = comment['label'].replace(['부정', '긍정'], [0, 1])

In [ ]:
comment.tail()

,data,label
96,작년에 프로그램A를 재밋게 봤던 시청자로서 올해의 미니드라마도 매우 기대가 됩니다....,1
97,프로그램C 잘 보았습니다. 모든일의 바탕은 안전인것 같습니다. 모두를 보호하는 최고...,1
98,위험한 시설에 대한 설명도 부탁드립니다,0
99,구체적으로 어떤 활동을 해왔었고 앞으로 어떤활동을 할건지 잘 설명해줬으면 좋았을 것...,0
100,우리 회사가 잘되면 나라도 잘된다. 회사 화이팅 나라 화이팅,1


In [ ]:
comment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 352 entries, 0 to 100
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    352 non-null    object
 1   label   352 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.2+ KB


In [ ]:
X = comment.data.to_list()  # array 형태는 에러가 발생하므로 리스트 형태로
y = comment.label.to_list()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)  # stratify=y: y를 기준으로 데이터를 균등하게 나눠

In [ ]:
len(X_train), len(y_train)

(281, 281)

In [ ]:
len(X_test), len(y_test)

(71, 71)

In [ ]:
X_train[:2]

['서비스가 차별화되고 좋은 것 같습니다.', '잘들었습니다. 다음 화도 기대하겠습니다. ']

In [ ]:
# Pre-Trained된 BERT tekenizer 가져오기
from transformers import BertTokenizerFast
bert_model = 'klue/bert-base'
tokenizer = BertTokenizerFast.from_pretrained(bert_model)

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
tokenizer.vocab_size

32000

In [ ]:
tokenizer.vocab

{'물리쳤': 31144,
 '만점': 10732,
 '이사': 4188,
 '혜진': 28720,
 '흥국': 10263,
 '스니커즈': 31269,
 '토록': 9193,
 '233': 29096,
 '대한항공': 9435,
 '기각': 10510,
 '바닷가': 12156,
 '비에스': 29464,
 '긍까': 26645,
 '법인세': 11100,
 '한성': 16024,
 '어두운': 8872,
 '중반': 7319,
 '640': 27291,
 '부르': 4681,
 '미디': 17043,
 '가속': 7717,
 '오묘': 26903,
 '##사회': 24618,
 '물놀이': 20107,
 '도달': 7466,
 '시청': 4479,
 '세심': 12541,
 '아낄': 28724,
 '고마워요': 20145,
 '앵무새': 28572,
 '##걱': 3229,
 '벅스': 12109,
 '더욱': 3940,
 '뷰스타': 28015,
 '먹일': 30173,
 '아쉽': 7985,
 '분향': 16593,
 '입각': 11848,
 '평신': 28120,
 '연화': 22973,
 '낙찰': 9259,
 '아끼': 7619,
 '베가': 30235,
 '더라고': 5626,
 'http': 8415,
 '共': 252,
 '대학가': 21321,
 '가꿔': 22975,
 '##어난': 4195,
 '감염자': 25837,
 '닥터': 11380,
 '이뤄지고': 12285,
 '장생': 25536,
 '컨테이너선': 24370,
 '時': 382,
 '따라오': 16022,
 '이청': 14289,
 '청주': 7959,
 '수프': 19985,
 '미분': 10747,
 '수중': 13165,
 '보잘것없': 29802,
 '##유리': 22910,
 '인맥': 11977,
 '전광판': 21218,
 '111': 13030,
 '##며': 2307,
 '##👉': 3349,
 '사각형': 25876,
 '함경': 23911,
 '시멘트

In [ ]:
X_train_enc = tokenizer(X_train, truncation=True, padding=True)
# truncation: vocab_size를 초과하는 길이를 갖는 문장은 자동으로 잘라내주기
# padding: 가장 긴 문장에 맞춰 알아서 패딩해주기

In [ ]:
X_test_enc = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
X_train_enc.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(X_train_enc['input_ids'][0])
# 정수 인덱싱 + 토큰화 + 패딩

[2, 3838, 2116, 4874, 2267, 2496, 2088, 1560, 2073, 575, 555, 2219, 3606, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(X_train_enc['token_type_ids'][13])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# token_type_ids: bert 모델에서 사용하는 feature, 문장이 두 개 이상 있을 경우 0과 1로 나타낸다
test = tokenizer('오늘 날씨가 좋다.', '내일도 좋을까?', return_tensors='pt')
# [CLS] 오늘 날씨가 좋다 [SEP] 내일도 좋을까?
# 토큰은 다른 값이 들어갈 수 있다
print(test['token_type_ids'])

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]])


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train_enc), y_train))
# tensor 형태의 데이터 셋으로 변환해줘야 한다
# 이미 X_train_enc는 dictionary 형태이지만 다시 한번 선언을 해줘서 에러를 미리 방지한다

In [ ]:
train_dataset = train_dataset.shuffle(1000).batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test_enc), y_test))
test_dataset = test_dataset.batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)
# shuffle을 하지 않는다 (모델 학습에도 영향을 주지 않고, 예측의 순서도 중요하지 않기 때문에 굳이 shuffle이 필요하지 않다)

In [ ]:
from transformers import TFBertForSequenceClassification
model = TFBertForSequenceClassification.from_pretrained(bert_model, num_labels=2, from_pt=True)
# 사전 학습된 모델인 TFBertForSequenceClassification을 huggingface에서 가져온 것이다
# num_labels=2: 이진 분류
# from_pt: 이전까지 pytorch로 작성되었던 내용을 모델을 불러오기 위해 tensorflow로 변환

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

In [ ]:
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])
# hf_compute_loss: 모델에 사전에 정의되어 있던 손실 함수를 불러온다

In [ ]:
model.fit(train_dataset, epochs=3, batch_size=16, validation_data=(test_dataset))  # 검증 데이터를 test_dataset 크기만큼

Epoch 1/3
18/18 [==============================] - 429s 21s/step - loss: 0.2793 - accuracy: 0.9110 - val_loss: 0.1189 - val_accuracy: 0.9577
Epoch 2/3
18/18 [==============================] - 351s 19s/step - loss: 0.0637 - accuracy: 0.9822 - val_loss: 0.2384 - val_accuracy: 0.9437
Epoch 3/3
18/18 [==============================] - 353s 20s/step - loss: 0.0253 - accuracy: 0.9929 - val_loss: 0.1839 - val_accuracy: 0.9437


In [ ]:
y_test_pred = model.predict(test_dataset)

5/5 [==============================] - 38s 6s/step


In [ ]:
y_test_pred.logits.shape

(71, 2)

In [ ]:
y_test_pred.logits[:10]
# 확률이 아닌 0에 가까운 정도, 1에 가까운 정도를 의미

array([[ 1.0766088, -1.1349989],
       [-2.66278  ,  2.8998063],
       [-2.2591672,  2.6280777],
       [ 3.022024 , -3.3898494],
       [-2.4975278,  2.7131014],
       [-2.735488 ,  3.0947516],
       [-2.5947554,  3.0302463],
       [ 2.561454 , -2.7052608],
       [-2.5373037,  2.9189374],
       [-2.886115 ,  3.1640806]], dtype=float32)

In [ ]:
df = pd.DataFrame(np.argmax(y_test_pred.logits, axis=1), columns=['predict'])
df

,predict
0,0
1,1
2,1
3,0
4,1
...,...
66,1
67,0
68,1
69,0


In [ ]:
df['true'] = y_test
df

,predict,true
0,0,1
1,1,1
2,1,1
3,0,0
4,1,1
...,...,...
66,1,1
67,0,0
68,1,1
69,0,0


In [ ]:
np.sum(df['true'] == df['predict'])/len(df)

0.9436619718309859